In [1]:
import os
import cv2
import tqdm
import json
import pickle
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from utils.dataset_utils import *
from collections import defaultdict
from scipy.optimize import linear_sum_assignment
from deep_sort_realtime.deepsort_tracker import DeepSort
from marl_aquarium.env.utils import scale

In [2]:
def scale_data(data):
    prey_pts = []
    pred_pts = None
    result = data["annotations"][0]["result"]
    for r in result:
        width, height = r["original_width"], r["original_height"]
        value = r["value"]
        x = (value["x"] / 100.0) * width
        y = (value["y"] / 100.0) * height
        labels = value.get("keypointlabels", [])
        label = labels[0] if labels else "Prey"
        if label == "Predator":
            pred_pts = (x, y)
        else:
            prey_pts.append((x, y))

    pred_arr = np.array([pred_pts])
    prey_arr = np.array(prey_pts)
    return pred_arr, prey_arr

def hungarian_assign(point_seq):
    ordered = [point_seq[0]]
    prev = point_seq[0]

    for t in range(1, len(point_seq)):
        current_point = point_seq[t]
        distance = np.linalg.norm(prev[:, None, :] - current_point[None, :, :], axis=2)
        _, col_ind = linear_sum_assignment(distance)
        curr_ord = current_point[col_ind]
        ordered.append(curr_ord)
        prev = curr_ord

    ordered = np.stack(ordered, axis=0)  # (T, N, 2)
    return ordered

def get_velocity(positions):
    velocities = []
    for i in range(1, len(positions)):
        velo = positions[i] - positions[i - 1]
        velocity = velo / 2 #every second frame got labeled
        velocities.append(velocity)
    return np.array(velocities)


def get_records(pred_ordered, prey_ordered, pred_velocities, prey_velocities):
    records = []
    T = pred_velocities.shape[0]
    M = prey_ordered.shape[1]

    for step in range(T):
        # Predator (immer Index 0)
        x  = float(pred_ordered[step, 0, 0])
        y  = float(pred_ordered[step, 0, 1])
        vx = float(pred_velocities[step, 0, 0])
        vy = float(pred_velocities[step, 0, 1])

        records.append({
            "frame": step,
            "label": "Predator",
            "conf": 1.0,
            "x": x, "y": y,
            "vx": vx, "vy": vy,
            "speed": float(math.hypot(vx, vy)),
            "angle": float(math.atan2(vy, vx)),
        })

        # Preys (Index 0..M-1)
        for i in range(M):
            x  = float(prey_ordered[step, i, 0])   # <-- step, i, ...
            y  = float(prey_ordered[step, i, 1])
            vx = float(prey_velocities[step, i, 0])
            vy = float(prey_velocities[step, i, 1])

            records.append({
                "frame": step,
                "label": "Prey",
                "conf": 1.0,
                "x": x, "y": y,
                "vx": vx, "vy": vy,
                "speed": float(math.hypot(vx, vy)),
                "angle": float(math.atan2(vy, vx)),
            })

    return records



def get_expert_features(frame, width, height, max_speed):    
    vscale = np.vectorize(scale)

    xs = np.array([float(np.asarray(det['x']).reshape(-1)[0]) for det in frame])
    ys = np.array([float(np.asarray(det['y']).reshape(-1)[0]) for det in frame])
    scaled_xs = vscale(xs, 0, width, 0, 1)
    scaled_ys = vscale(ys, 0, height, 0, 1)

    vxs = np.array([float(np.asarray(det['vx']).reshape(-1)[0]) for det in frame])
    vys = np.array([float(np.asarray(det['vy']).reshape(-1)[0]) for det in frame])

    thetas = np.array([det['angle'] for det in frame])
    scaled_thetas = vscale(thetas, -np.pi, np.pi, 0, 1)

    cos_t = np.cos(thetas)                        
    sin_t = np.sin(thetas)

    # pairwise distances
    dx = scaled_xs[None, :] - scaled_xs[:, None]
    dy = scaled_ys[None, :] - scaled_ys[:, None]

    # relative velocities
    rel_vx = cos_t[:, None] * vxs[None, :] + sin_t[:, None] * vys[None, :]
    rel_vy = -sin_t[:, None] * vxs[None, :] + cos_t[:, None] * vys[None, :]

    rel_vx = np.clip(rel_vx, -max_speed, max_speed)
    rel_vy = np.clip(rel_vy, -max_speed, max_speed)
    
    scaled_rel_vx = vscale(rel_vx, -max_speed, max_speed, -1, 1)
    scaled_rel_vy = vscale(rel_vy, -max_speed, max_speed, -1, 1)

    n = scaled_xs.shape[0]
    thetas_mat = np.tile(scaled_thetas[:, None], (1, n))
    features = np.stack([dx, dy, scaled_rel_vx, scaled_rel_vy, thetas_mat], axis=-1)

    mask = ~np.eye(n, dtype=bool) # shape (N, N)
    neigh = features[mask].reshape(n, n-1, 5)

    pred_tensor = torch.from_numpy(neigh[0]).unsqueeze(0)
    prey_tensor = torch.from_numpy(neigh[1:]) # shape (N-1, N-1, 5)

    return pred_tensor, prey_tensor


def get_expert_tensors(records, max_speed):
    preds = []
    preys = []

    frame_ids = sorted({rec["frame"] for rec in records})
    for frame_idx in frame_ids:
        frame = [rec for rec in records if rec["frame"] == frame_idx]
        if not frame:
            continue
        pred_tensor, prey_tensor = get_expert_features(frame, width=2160, height=2160, max_speed=max_speed)
        preds.append(pred_tensor)
        preys.append(prey_tensor)

    pred_tensor = torch.stack(preds, dim=0)
    prey_tensor = torch.stack(preys, dim=0)

    return pred_tensor, prey_tensor

In [3]:
def extract_windows(tensor, window_len):
    n_frames = tensor.shape[0]
    windows = []

    for start in range(0, n_frames - window_len + 1):
        windows.append(tensor[start:start + window_len])

    return torch.stack(windows, dim=0)

In [4]:
data_folder = rf'..\data\1. Data Processing\raw\pred_attack'
tensor_folder = rf'..\data\1. Data Processing\processed\video\expert_tensors\windows'
window_len = 10

pred_tensors_list = []
prey_tensors_list = []

for file in os.listdir(data_folder):
    if file.endswith(".json"):
        file_path = os.path.join(data_folder, file)
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        pairs = [scale_data(pts) for pts in data]
        pred_pts = [p for p, _ in pairs] 
        prey_pts = [q for _, q in pairs]

        pred_ordered = hungarian_assign(pred_pts)
        prey_ordered = hungarian_assign(prey_pts)

        pred_velocities = get_velocity(pred_ordered) #vx, vy
        prey_velocities = get_velocity(prey_ordered)

        records = get_records(pred_ordered, prey_ordered, pred_velocities, prey_velocities)

        pred_tensor, prey_tensor = get_expert_tensors(records, max_speed=10)

        pred_windows = extract_windows(pred_tensor, window_len=window_len)
        prey_windows = extract_windows(prey_tensor, window_len=window_len)

        pred_tensors_list.append(pred_windows)
        prey_tensors_list.append(prey_windows)

pred_tensors = torch.cat(pred_tensors_list, dim=0)
prey_tensors = torch.cat(prey_tensors_list, dim=0)

n, window, agents, neighs, feature = prey_tensors.shape
flag = torch.zeros((n, window, agents, neighs, 1), dtype=prey_tensors.dtype, device=prey_tensors.device)
flag[:, :1, 0] = 1
prey_tensors = torch.cat([flag, prey_tensors], dim=-1)

pred_path = os.path.join(tensor_folder, f"{window_len} windows" , f"pred_tensors_hl_w{window_len}_n{len(pred_tensors)}.pkl")
prey_path = os.path.join(tensor_folder, f"{window_len} windows" , f"prey_tensors_hl_w{window_len}_n{len(prey_tensors)}.pkl")

torch.save(pred_tensors, pred_path)
torch.save(prey_tensors, prey_path)

print("Pred Tensors Shape:", tuple(pred_tensors.shape))
print("Prey Tensors Shape:", tuple(prey_tensors.shape))

Pred Tensors Shape: (769, 10, 1, 32, 5)
Prey Tensors Shape: (769, 10, 32, 32, 6)
